In [1]:
#!/usr/bin/env python

from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster()
cluster.adapt(maximum=20)

cluster

In [2]:

client = Client(cluster)

client

Client Scheduler: tcp://10.32.5.10:32989 Dashboard: /user/0000-0001-7783-5629/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
import xarray as xr
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client
import numpy as np                                                                                        
import zarr


In [4]:
import sys, glob
import numpy as np
import xarray as xr
import xscale 
import xscale.spectral.fft as xfft
import Wavenum_freq_spec_func as wfs
import time
import gcsfs

import matplotlib.pyplot as plt


In [5]:
%time

fs = gcsfs.GCSFileSystem(project='pangeo-181919', token='anon', access='read_only')
gcs = gcsfs.GCSFileSystem(gcs=fs,check=False,create=False)

mapzarru = gcs.get_mapper('pangeo-data/eNATL60-BLBT02-SSU-1h')
dsu = xr.open_zarr(mapzarru)

mapzarrv = gcs.get_mapper('pangeo-data/eNATL60-BLBT02-SSV-1h')
dsv = xr.open_zarr(mapzarrv)



CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs


In [6]:
dsu

<xarray.Dataset>
Dimensions:              (axis_nbounds: 2, time_counter: 8760, x: 8354, y: 4729)
Coordinates:
    nav_lat              (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    nav_lon              (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    time_centered        (time_counter) datetime64[ns] dask.array<chunksize=(744,), meta=np.ndarray>
  * time_counter         (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2010-06-30T23:30:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    sozocrtx             (time_counter, y, x) float32 dask.array<chunksize=(24, 120, 120), meta=np.ndarray>
    time_counter_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(744, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    NCO:          4.4.6
    TimeStamp:    08/01/2019 09:34:23 +0100
    description:  ocean U grid variables
    file_name:    eNATL60-BLBT02_1h_20090630_20090704_gridU-2D_200907

In [7]:
%%time
lat=dsu['nav_lat']
lon=dsu['nav_lon']
 
# definition of the area

latmin = 40.0; latmax = 41.0;
lonmin = -40.0; lonmax = -39.0;

domain = (lonmin<lon) * (lon<lonmax) * (latmin<lat) * (lat<latmax)
where = np.where(domain)

#get indice
jmin = np.min(where[0][:])
jmax = np.max(where[0][:])
imin = np.min(where[1][:])
imax = np.max(where[1][:])

latbox=lat[jmin:jmax,imin:imax]
lonbox=lon[jmin:jmax,imin:imax]


CPU times: user 49.3 s, sys: 5.38 s, total: 54.6 s
Wall time: 8min 15s


In [8]:
%%time

print('Select dates')
u_JFM=dsu.sel(time_counter=slice('2010-01-01','2010-03-31'))['sozocrtx']
v_JFM=dsv.sel(time_counter=slice('2010-01-01','2010-03-31'))['somecrty']




Select dates
CPU times: user 485 ms, sys: 174 ms, total: 659 ms
Wall time: 689 ms


In [ ]:
check_NaN = (1*np.isnan(u_JFM.values)).sum()
print(check_NaN)

In [ ]:
check_NaN = (1*np.isnan(u_JFM.values)).sum()
print(check_NaN)